In [2]:
import vectorbt as vbt
print("vectorbt version:", vbt.__version__)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

vectorbt version: 0.27.3


In [3]:
from utils.eda_utils import dataset_info_summary, system_info
from data.Data_PipLine import data_from_csv
from strategy.SMACrossoverStrategy import SMACrossoverStrategy

In [4]:
QQU = r"C:\Users\wzhou\Desktop\Model\data\qqu\QQU ETF Stock Price History.csv"
df = data_from_csv(QQU)
price = df['Close']

In [5]:
sma = SMACrossoverStrategy(price, fast_window=9, slow_window=20)
sma.run()
sma.backtest()
sma.plot().show()